# Google Cloud Parallel Data Read Speeds with Dask

This benchmarking was inspired by the [Abernathey et al. (2021)](https://www.computer.org/csdl/magazine/cs/2021/02/09354557/1reXu4gJjri) paper, which includes a detailed analysis and discussion of cloud-native big scientific data. The paper is much wider in scope than this benchmarking, so it is recommended that a reader unacclimated with this subject matter first studies its information before going through this notebook.

Unfortunately, due to the access token required to retrieve data the Google Cloud Storage Bucket used in this benchmarking, the public reader will not be able to run this notebook on their own. If you wish to download your own copy and alter the code, the test data from [NOAA's ETOPO1 Global Relief Model](https://www.ngdc.noaa.gov/mgg/global/relief/ETOPO1/data/ice_surface/grid_registered/netcdf/) can be found here. A file transformation notebook is also found within the repository, which documents my methods for converting from the original NetCDF format. Note that a CSV version was created outside of this project space.

Different methods for accessing Google Cloud Storage are used--contigent on the file type of the data. Though discussed in further detail later on, I encourage the reader to read about the different file formats used in this benchmarking, as well as the access APIs. In many instances, data can be loaded directly into Dask objects without further manipulation. Other libraries were also used to load data in, mainly due to the limitations of Dask or speed benefits of using another library.

A significant portion of the code used in this benchmarking is directly taken from [Ryan Abernathey's demonstration notebook](https://github.com/earthcube2020/ec20_abernathey_etal/blob/master/cloud_storage.ipynb), and it is worth viewing before beginning this read.

## Imports & Client Initialization

In [ ]:
import dask.array as dsa
import fsspec
import numpy as np
import dask.dataframe as dd
from contextlib import contextmanager
import xarray as xr
import intake
import time
import dask
from matplotlib import pyplot as plt
import pandas as pd

In [ ]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)

## Benchmarking Setup

First, we will create this null storage object. To measure our throughput, all of the data will need to be accessed at a single time and can be achieved by storing the data into this null storage target.

In [ ]:
class DevNullStore:
    def __init__(self):
        pass
    def __setitem__(*args, **kwargs):
        pass

null_store = DevNullStore()

The Diagnostic Timer will keep track of data retrieval times and store them within a pandas dataframe for later processing and analysis.

In [ ]:
class DiagnosticTimer:
    def __init__(self):
        self.diagnostics = []
        
    @contextmanager
    def time(self, **kwargs):
        tic = time.time()
        yield
        toc = time.time()
        kwargs["runtime"] = toc - tic
        self.diagnostics.append(kwargs)
        
    def dataframe(self):
        return pd.DataFrame(self.diagnostics)
    
diag_timer = DiagnosticTimer()

This naming function will keep track of our read throughput for each file type and make it easier to plot all of the cases.

In [ ]:
def name(fileType, daf): # Takes a string fileType input & dataframe type input for daf 
    globals()[f"df_{fileType}"] = daf
    global df, da
    del df, da
    diag_timer.diagnostics = []

## Perform Benchmarking

Though we are accessing the data from different formats, the core process will be the exact same. Dask is lazy by default, which means we will be using the previously defined null_store to measure throughput. Converting from DataFrames to Arrays, for example, will not affect the access speed because the data is not actually read from the source until we "store" it. So, these read speeds should be the same whether you are initially pointing to data using a dataframe or array.

In [3]:
# A token may or may not be needed depending on the permissions your specific Google Bucket requires.
token = '/home/ubuntu/Cloud-Data-Transfer-Speed-Benchmarks/cloud-data-benchmarks.json'

Information about our cluster is also collected during the tests. Importantly, the number of parallel reads will correspond to the total number of workers.

In [ ]:
def total_nthreads():
    return sum([v for v in client.nthreads().values()])

def total_ncores():
    return sum([v for v in client.ncores().values()])

def total_workers():
    return len(client.ncores())

### CSV

This is the classic tabular data format. Students & professionals alike use this format due to its simplicity and flexibility bewteen applications. As you will see, this format has the slowest read times of all being tested but is included in the benchmarking given its widespread use.

#### Single File

In [ ]:
tic1 = time.time()
df0 = dd.read_csv('gs://cloud-data-benchmarks/ETOPO1_Ice_g_gmt4.csv', storage_options={'token':token}, assume_missing=True)
toc1 = time.time()
connectTime = toc1-tic1

da = df0.to_dask_array(lengths=True)
del df0
chunksize = np.prod(da.chunksize) * da.dtype.itemsize
da

This is the main loop. A few modifications have been made from Abernathey's original version, but most notably we want to measure the connection time to the data reference. This shouldn't matter if we are purely measuring access times, but it is important when we use different modules to connect to Google Cloud Storage.

In [ ]:
diag_kwargs = dict(nbytes=da.nbytes, chunksize=chunksize, format='csv')

for nworkers in [1, 3, 4]:
    cluster.scale(nworkers)
    time.sleep(10)
    client.wait_for_workers(nworkers)
    print(nworkers)
    with diag_timer.time(nthreads=total_nthreads(), ncores=total_ncores(), nworkers=total_workers(), connectTime=connectTime, **diag_kwargs):
        future = dsa.store(da, null_store, lock=False, compute=False)
        dask.compute(future, retries=5)
    del future

In [ ]:
df = diag_timer.dataframe()
df['throughput_Mbps'] = da.nbytes / 1e6 / df['runtime']
name('csv', df)
df_csv

#### Multiple Files

While one CSV file is the most common way one will see data of this format presented, we are interested in determining if partitioning (splitting up) the data into file sizes automatically determined by Dask will speed up the read from cloud storage. The process is the exact same, with the exception of a minor amount of required preprocessing.

In [ ]:
tic1 = time.time()
df0 = dd.read_csv('gs://cloud-data-benchmarks/csvpartitions/*', storage_options={'token':token}, assume_missing=True)
toc1 = time.time()
connectTime = toc1-tic1

da = df0.to_dask_array(lengths=True)
del df0
chunksize = np.prod(da.chunksize) * da.dtype.itemsize
da

In [ ]:
diag_kwargs = dict(nbytes=da.nbytes, chunksize=chunksize, format='part_csv')

for nworkers in [1, 3, 4]:
    cluster.scale(nworkers)
    time.sleep(10)
    client.wait_for_workers(nworkers)
    print(nworkers)
    with diag_timer.time(nthreads=total_nthreads(), ncores=total_ncores(), nworkers=total_workers(), connectTime=connectTime, **diag_kwargs):
        future = dsa.store(da, null_store, lock=False, compute=False)
        dask.compute(future, retries=5)
    del future

In [ ]:
df = diag_timer.dataframe()
df['throughput_Mbps'] = da.nbytes / 1e6 / df['runtime']
name('partcsv', df)
df_partcsv

### NetCDF

NetCDF files are extremely common in geospatial and climate science, which is the original format of the test data used in this benchmarking. Data from these fields are commonly very large, with sizes only increasing as missions collecting samples at higher resolutions are conducted. The informed scientist should have a good grasp on the speed limitations of this format and the best ways to load in and perform computations within Python.

We load the NetCDF using the intake library. This is a great way to load the data from GCS without directly using the gcsfs module. Depending on the NetCDF file contents, decoding the data using the engines included with xarray.open_dataset(...) can be troublesome. Read the XArray documentation about accessing files using xarray.open_dataset(...) [here](https://docs.xarray.dev/en/stable/user-guide/io.html).

The intake module is much easier to use, and offers some additional functionality in the form of catalogs. Read about catalogs and their uses [here](https://intake.readthedocs.io/en/latest/catalog.html).

In [ ]:
tic1 = time.time()
data = intake.open_netcdf('gs://cloud-data-benchmarks/ETOPO1_Ice_g_gmt4.nc').to_dask()
toc1 = time.time()
connectTime = toc1-tic1
data = data.to_array() # Changes from DataSet to DataArray
da = data.data # Retrieves raw values from wrapped Xarray DataArray object
da = dsa.from_array(da)
chunksize = np.prod(da.chunksize) * da.dtype.itemsize
da

We encounter a large unmanaged memory warning in the dask.compute(future, retries=5) line, causing the data store to fail. Investigation into a solution is underway, but please do not hesistate to comment with any suggestions.

In [ ]:
diag_kwargs = dict(nbytes=da.nbytes, chunksize=chunksize, format='NetCDF')

for nworkers in [1, 3, 4]:
    cluster.scale(nworkers)
    time.sleep(10)
    client.wait_for_workers(nworkers)
    print(nworkers)
    with diag_timer.time(nthreads=total_nthreads(), ncores=total_ncores(), nworkers=total_workers(), connectTime=connectTime, **diag_kwargs):
        future = dsa.store(da, null_store, lock=False, compute=False)
        dask.compute(future, retries=5)
    del future

In [ ]:
df = diag_timer.dataframe()
df['throughput_Mbps'] = da.nbytes / 1e6 / df['runtime']
name('netcdf', df)
df_netcdf

### Parquet

Throughput will be tested in two different ways:
* A CSV file was partitioned into a DataFrame by Dask, and 1 parquet file was written per DataFrame partition. All of these files will be read into the null storage target
* A single parquet file containing all of the original CSV information will be read into the null storage target

#### Multiple Files

In [ ]:
tic1 = time.time()
df0 = dd.read_parquet("gs://cloud-data-benchmarks/parquetpartitions/*")
toc1 = time.time()
connectTime = toc1 - tic1

da = df0.to_dask_array(lengths=True)
del df0
chunksize = np.prod(da.chunksize) * da.dtype.itemsize
da

In [ ]:
diag_kwargs = dict(nbytes=da.nbytes, chunksize=chunksize, format='part_parquet')

for nworkers in [1, 3, 4]:
    cluster.scale(nworkers)
    time.sleep(10)
    client.wait_for_workers(nworkers)
    print(nworkers)
    with diag_timer.time(nthreads=total_nthreads(), ncores=total_ncores(), nworkers=total_workers(), connectTime=connectTime, **diag_kwargs):
        future = dsa.store(da, null_store, lock=False, compute=False)
        dask.compute(future, retries=5)
    del future

In [ ]:
df = diag_timer.dataframe()
df['throughput_Mbps'] = da.nbytes / 1e6 / df['runtime']
name('partparquet', df)

#### Single File

We encounter a problem with large unmanaged memory when attempting to compute the size of DataFrame. Generally, it is recommended that parquet files be partitioned in advance to avoid this issue. 

In [ ]:
tic1 = time.time()
df0 = dd.read_parquet("gs://cloud-data-benchmarks/ETOPO1_Ice_g_gmt4.parquet")
toc1 = time.time()
connectTime = toc1 - tic1

# Memory Problem between this print statements, similar to the NetCDF issue
print("Calculating size of array")
da = df0.to_dask_array(lengths=True)
print("Size of array has been calculated")

del df0
chunksize = np.prod(da.chunksize) * da.dtype.itemsize
da

In [ ]:
diag_kwargs = dict(nbytes=da.nbytes, chunksize=chunksize, format='parquet')

for nworkers in [1, 3, 4]:
    cluster.scale(nworkers)
    time.sleep(10)
    client.wait_for_workers(nworkers)
    print(nworkers)
    with diag_timer.time(nthreads=total_nthreads(), ncores=total_ncores(), nworkers=total_workers(), connectTime=connectTime, **diag_kwargs):
        future = dsa.store(da, null_store, lock=False, compute=False)
        dask.compute(future, retries=5)
    del future

In [ ]:
df = diag_timer.dataframe()
df['throughput_Mbps'] = da.nbytes / 1e6 / df['runtime']
name('parquet', df)
df_parquet

### Zarr

See <a href="https://www.programcreek.com/python/example/128207/dask.array.from_zarr#:~:text=.chunks)-,Example%205,-Project%3A%20napari">this python example</a> of dask.array.from_zarr(...) for further understanding of the differences between a Zarr Array & Zarr Group.

A Zarr Group is a very good choice of storage utility for multiscale data.

#### Zarr Array

In [ ]:
tic1 = time.time()
da = dsa.from_zarr('gs://cloud-data-benchmarks/ETOPO1_Ice_g_gmt4.zarray')
toc1 = time.time()
connectTime = toc1 - tic1

chunksize = np.prod(da.chunksize) * da.dtype.itemsize
da

In [ ]:
diag_kwargs = dict(nbytes=da.nbytes, chunksize=chunksize, format='zarr_array')

for nworkers in [1, 3, 4]:
    cluster.scale(nworkers)
    time.sleep(10)
    client.wait_for_workers(nworkers)
    print(nworkers)
    with diag_timer.time(nthreads=total_nthreads(), ncores=total_ncores(), nworkers=total_workers(), connectTime=connectTime, **diag_kwargs):
        future = dsa.store(da, null_store, lock=False, compute=False)
        dask.compute(future, retries=5)
    del future

In [ ]:
df = diag_timer.dataframe()
df['throughput_Mbps'] = da.nbytes / 1e6 / df['runtime']
name('zarray', df)

#### Zarr Hierachical Group

Xarray provides a fantasic tool in their API for opening Zarr groups. By consolidating the metadata upon creation of the group, the read speed increases considerably more than if it was left in its base state.

In [ ]:
tic1 = time.time()
zarr_ds = xr.open_zarr(store='gs://cloud-data-benchmarks/ETOPO1_Ice_g_gmt4.zarr', storage_options={'token':token}, consolidated=True)
toc1 = time.time()
connectTime = toc1-tic1

darray = zarr_ds.to_array()
da = darray.data

chunksize = np.prod(da.chunksize) * da.dtype.itemsize
da

In [ ]:
diag_kwargs = dict(nbytes=da.nbytes, chunksize=chunksize, format='zarr_group')

for nworkers in [1, 3, 4]:
    cluster.scale(nworkers)
    time.sleep(10)
    client.wait_for_workers(nworkers)
    print(nworkers)
    with diag_timer.time(nthreads=total_nthreads(), ncores=total_ncores(), nworkers=total_workers(), connectTime=connectTime, **diag_kwargs):
        future = dsa.store(da, null_store, lock=False, compute=False)
        dask.compute(future, retries=5)
    del future

In [ ]:
df = diag_timer.dataframe()
df['throughput_Mbps'] = da.nbytes / 1e6 / df['runtime']
name('zgroup', df)
df_zgroup

## Plots

In [ ]:
ax = df_csv.plot(x='nworkers', y='throughput_Mbps', title='Cloud Data Read Speeds with Dask', kind='scatter', color='green')
plt.grid(True)
df_partcsv.plot(ax=ax, x='nworkers', y='throughput_Mbps', kind='scatter', color='limegreen')
df_partparquet.plot(ax=ax, x='nworkers', y='throughput_Mbps', kind='scatter', color='blue')
df_zarray.plot(ax=ax, x='nworkers', y='throughput_Mbps', kind='scatter', color='red')
df_zgroup.plot(ax=ax, x='nworkers', y='throughput_Mbps', kind='scatter', 
                 xlabel='Number of Parallel Reads', ylabel='Throughput (Mbps)', color='darkred')
plt.grid(True)
plt.legend(['CSV', 'Partitioned CSV', 'Partitioned Parquet','Zarr Array', 'Zarr Group'], loc='upper left', title='Store Formats')
plt.show()